In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
# Loading the dataset
def loaddata(filename):
    df = pd.read_csv(f'/content/drive/MyDrive/data-brm/{filename}.csv', sep=',', encoding='latin-1')
    return df

def loaddata2():
    df = pd.read_csv(f'/content/drive/MyDrive/cleaned_book_data.csv', sep=',', encoding='latin-1')
    return df

books   = loaddata2()
ratings = loaddata("ratings")

In [ ]:
books.shape

(10000, 11)

In [ ]:
import torch

!pip uninstall torch-scatter torch-sparse torch-geometric torch-cluster  --y
!pip install torch-scatter -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install torch-cluster -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install git+https://github.com/pyg-team/pytorch_geometric.git

Looking in links: https://data.pyg.org/whl/torch-2.0.1+cu118.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 43.8 MB/s eta 0:00:00
Looking in links: https://data.pyg.org/whl/torch-2.0.1+cu118.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 50.7 MB/s eta 0:00:00
Looking in links: https://data.pyg.org/whl/torch-2.0.1+cu118.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 51.7 MB/s eta 0:00:00
  Cloning https://github.com/pyg-team/pytorch_geometric.git to /tmp/pip-req-build-hwfezv9e
  Running command git clone --filter=blob:none --quiet https://github.com/pyg-team/pytorch_geometric.git /tmp/pip-req-build-hwfezv9e
  Resolved https://github.com/pyg-team/pytorch_geometric.git to commit 9e687b412e87aa0f07ce7152e31d6f40271ed86d
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for torch_geometric: filename=torch_geometric-2.4.0-py3-none-any.whl size=99

In [ ]:
# import required modules
import random
from tqdm.notebook import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import model_selection, metrics, preprocessing
import copy

import torch
from torch import nn, optim, Tensor

from torch_sparse import SparseTensor, matmul

from collections import defaultdict

from torch_geometric.utils import structured_negative_sampling
from torch_geometric.data import download_url, extract_zip
from torch_geometric.nn.conv.gcn_conv import gcn_norm
from torch_geometric.nn.conv import MessagePassing
from torch_geometric.typing import Adj
import torch.nn.functional as F

Dataset already loaded. now preprocessing

In [ ]:
rating_users = ratings['user_id'].value_counts().reset_index().\
               rename({'index':'user_id','user_id':'rating'}, axis=1)

In [ ]:
rating_books = ratings['book_id'].value_counts().reset_index().\
               rename({'index':'book_id','book_id':'rating'}, axis=1)

In [ ]:
ratings = ratings[ratings['user_id'].isin(rating_users[rating_users['rating']>5]['user_id'])]
ratings = ratings[ratings['book_id'].isin(rating_books[rating_books['rating']> 2000]['book_id'])]

In [ ]:
temp = ratings.head(650000)
ratings = temp

In [ ]:
ratings.shape

(650000, 3)

In [ ]:
# Using LabelEncoder to convert categorical user/book id to numerical ids
lbl_user = preprocessing.LabelEncoder()
lbl_books = preprocessing.LabelEncoder()

# converting user/book id into numerical ids
ratings.user_id = lbl_user.fit_transform(ratings.user_id.values)
ratings.book_id = lbl_books.fit_transform(ratings.book_id.values)

In [ ]:
print(lbl_user)
print(lbl_books)
print(ratings.user_id)
print(ratings.book_id)

In [ ]:
ratings.head(20)

In [ ]:
print(ratings.user_id.max())
print(ratings.book_id.max())

16270
505


In [ ]:
# load data from the dataframe and convert it into a form suitable for graph-recommender system
# src_index_col, dst_index_col, link_index_col represents the column names of user id, book id and rating
def load_edge_csv(df,
                  src_index_col,
                  dst_index_col,
                  link_index_col,
                  rating_threshold=3.5):
    """Loads csv containing edges between users and items

    Args:
        path (str): path to csv file
        src_index_col (str): column name of users
        src_mapping (dict): mapping between row number and user id
        dst_index_col (str): column name of items
        dst_mapping (dict): mapping between row number and item id
        link_index_col (str): column name of user item interaction
        rating_threshold (int, optional): Threshold to determine positivity of edge. Defaults to 4.

    Returns:
        torch.Tensor: 2 by N matrix containing the node ids of N user-item edges
    """

    edge_index = None     #this will hold edge indices
    src = [user_id for user_id in  df['user_id']]   #stores user id's as list

    num_users = len(df['user_id'].unique())         #number of  unique user id's

    dst = [(book_id) for book_id in df['book_id']]   #stores book id's as list

    link_vals = df[link_index_col].values     #stores the ratings between user and books as list


    # binary attribute(0/1) indicating whether the rating is greater than or equal to a rating_threshold.
    #helps determine whether edge is positive or not
    edge_attr = torch.from_numpy(df[link_index_col].values).view(-1, 1).to(torch.long) >= rating_threshold

    edge_values = []

    # The code creates the edge_index list of lists, where edge_index[0]
    # contains the user IDs (source nodes) of the edges, and edge_index[1]
    # contains the item IDs (destination nodes) of the edges.
    # These edges are determined based on the edge_attr values.
    edge_index = [[], []]


    #add values to edge_index and edge_values.
    # edge_index -> 0: user_id and 1:book_id
    # edge_values -> ratings
    # only enters the loop if edge_attr element is a positive entry
    for i in range(edge_attr.shape[0]):
        if edge_attr[i]:
            edge_index[0].append(src[i])
            edge_index[1].append(dst[i])
            edge_values.append(link_vals[i])

    # edge_values is the label we will use for compute training loss
    return edge_index, edge_values

In [ ]:

edge_index, edge_values = load_edge_csv(
    ratings,
    src_index_col='user_id',
    dst_index_col='book_id',
    link_index_col='rating',
    rating_threshold=1)

# print("0: ",edge_index[0][:20])
# print("1: ",edge_index[1])
# print("values: ",edge_values)


In [ ]:
# Convert to tensor
# We use LongTensor here because the .propagate() method in the model needs either LongTensor or SparseTensor
print("0: ",edge_index[0][:20])
print("1: ",edge_index[1][:20])
print("values: ",edge_values[:20])

edge_index = torch.LongTensor(edge_index)
edge_values = torch.tensor(edge_values)

print(edge_index)
print(edge_index.size())

print(edge_values)
print(edge_values.size())

0:  [0, 1, 1, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]
1:  [244, 25, 32, 278, 68, 248, 347, 17, 26, 20, 1, 22, 23, 101, 241, 34, 265, 307, 25, 82]
values:  [5, 4, 4, 5, 4, 3, 4, 5, 5, 5, 5, 5, 5, 5, 2, 5, 3, 4, 3, 4]
tensor([[    0,     1,     1,  ..., 11286, 15204, 15299],
        [  244,    25,    32,  ...,   178,    42,    16]])
torch.Size([2, 650000])
tensor([5, 4, 4,  ..., 3, 5, 4])
torch.Size([650000])


In [ ]:
ratings.shape

(650000, 3)

In [ ]:
# num_users, num_books = len(user_mapping), len(book_mapping)

num_users = len(ratings['user_id'].unique())
num_books = len(ratings['book_id'].unique())

print(f"num_users {num_users}, num_books {num_books}")

num_users 16271, num_books 506


In [ ]:
# function to construct an adjacency matrix
def convert_r_mat_edge_index_to_adj_mat_edge_index(input_edge_index, input_edge_values):

    # R is initialised with 0's of size num_users x num_books
    R = torch.zeros((num_users, num_books))
    for i in range(len(input_edge_index[0])):
        row_idx = input_edge_index[0][i]      #source
        col_idx = input_edge_index[1][i]      #destination
        R[row_idx][col_idx] = input_edge_values[i] # assign actual edge value to Interaction Matrix

    # construct R transpose
    R_transpose = torch.transpose(R, 0, 1)

    # create adj_matrix
    adj_mat = torch.zeros((num_users + num_books , num_users + num_books))

    # upper left quedrant is filled with R
    adj_mat[: num_users, num_users :] = R.clone()

    # bottom right quadrant if filled with R_transpose
    adj_mat[num_users :, : num_users] = R_transpose.clone()

    # converts the adjacency matrix to sparse matrix
    # this saves memory space by storing only non-zero values
    adj_mat_coo = adj_mat.to_sparse_coo()

    # stores indices of non-zero elements
    adj_mat_coo_indices = adj_mat_coo.indices()

    # stores the non-zero rating values
    adj_mat_coo_values = adj_mat_coo.values()

    # returns both
    return adj_mat_coo_indices, adj_mat_coo_values

In [ ]:
# convert adjacency matrix to original matrix form
def convert_adj_mat_edge_index_to_r_mat_edge_index(input_edge_index, input_edge_values):

  # Ensure input_edge_index and input_edge_values are on the same device as the model
    input_edge_index = input_edge_index.to(device)
    input_edge_values = input_edge_values.to(device)

   # Create a sparse tensor from input_edge_index and input_edge_values
    sparse_input_edge_index = torch.sparse_coo_tensor(input_edge_index, input_edge_values)

    # Convert the sparse tensor to a dense tensor (adjacency matrix)
    adj_mat = sparse_input_edge_index.to_dense()

    # Extract the interaction matrix from the adjacency matrix
    interact_mat = adj_mat[:num_users, num_users:]

    # Convert the interaction matrix to a sparse tensor
    r_mat_edge_index = interact_mat.to_sparse().indices()
    r_mat_edge_values = interact_mat.to_sparse().values()

    return r_mat_edge_index, r_mat_edge_values

In [ ]:
edge_index.shape

torch.Size([2, 650000])

In [ ]:
num_interactions = edge_index.shape[1]
all_indices = [i for i in range(num_interactions)]

# splitting the datas for training, testing and validation by finding its indices. This finds the index range(columns) from the 2 x n edge_index list
train_indices, test_indices = train_test_split(all_indices,
                                               test_size=0.2,
                                               random_state=1)

val_indices, test_indices = train_test_split(test_indices,
                                             test_size=0.5,
                                             random_state=1)

In [ ]:


train_edge_index = edge_index[:, train_indices]
train_edge_value = edge_values[train_indices]

val_edge_index = edge_index[:, val_indices]
val_edge_value = edge_values[val_indices]

test_edge_index = edge_index[:, test_indices]
test_edge_value = edge_values[test_indices]

In [ ]:
print(train_edge_index)
print(train_edge_value)
print(val_edge_index)
print(val_edge_value)
print(test_edge_index)
print(test_edge_value)


In [ ]:
print(f"num_users {num_users}, num_books {num_books}, num_interactions {num_interactions}")
print(f"train_edge_index {train_edge_index}")
print((num_users + num_books))
print(torch.unique(train_edge_index[0]).size())
print(torch.unique(train_edge_index[1]).size())

print(test_edge_value)
print(test_edge_value.size())
print(train_edge_value.size())

num_users 15059, num_books 505, num_interactions 600000
train_edge_index tensor([[ 3791,  1595, 14385,  ..., 12618,  8281,  5520],
        [   66,   265,    49,  ...,   195,   131,    92]])
15564
torch.Size([15003])
torch.Size([505])
tensor([5, 4, 5,  ..., 5, 5, 3])
torch.Size([60000])
torch.Size([480000])


In [ ]:
# converting regular matrix into adjacency matrix

train_edge_index, train_edge_values  = convert_r_mat_edge_index_to_adj_mat_edge_index(train_edge_index, train_edge_value)
val_edge_index, val_edge_values = convert_r_mat_edge_index_to_adj_mat_edge_index(val_edge_index, val_edge_value)
test_edge_index, test_edge_values = convert_r_mat_edge_index_to_adj_mat_edge_index(test_edge_index, test_edge_value)

In [ ]:
print(train_edge_index)
print(train_edge_index.size())
print(val_edge_index)
print(val_edge_index.size())
print(test_edge_index)
print(test_edge_index.size())

print(f"\n train_edge_values: \n {train_edge_values} \n {train_edge_values.size()}")
print(f"\n val_edge_values: \n {val_edge_values} \n {val_edge_values.size()}")
print(f"\n test_edge_values: \n {test_edge_values} \n {test_edge_values.size()}")

Implementing LigthGCN

In [ ]:
# defines LightGCN model
class LightGCN(MessagePassing):
    """LightGCN Model as proposed in https://arxiv.org/abs/2002.02126
    """

    # embedding_dim : each user/item will be represented in a 32 dimensional vector. The embedding vectors capture the
                    # latent characteristics or features of users and items
    def __init__(self, num_users, num_items, embedding_dim=32, K=3, add_self_loops=False, dropout_rate=0.1):
        """Initializes LightGCN Model

        Args:
            num_users (int): Number of users
            num_items (int): Number of items
            embedding_dim (int, optional): Dimensionality of embeddings. Defaults to 8.
            K (int, optional): Number of message passing layers. Defaults to 3.
            add_self_loops (bool, optional): Whether to add self loops for message passing. Defaults to False.
        """
        super().__init__()
        self.dropout_rate = dropout_rate
        self.num_users = num_users
        self.num_items = num_items
        self.embedding_dim = embedding_dim
        self.K = K
        self.add_self_loops = add_self_loops


        # define user and item embedding for direct look up.
        # embedding dimension: num_user/num_item x embedding_dim
        self.users_emb = nn.Embedding(num_embeddings=self.num_users, embedding_dim=self.embedding_dim) # e_u^0

        self.items_emb = nn.Embedding(num_embeddings=self.num_items, embedding_dim=self.embedding_dim) # e_i^0


        # "Fills the input Tensor with values drawn from the normal distribution"
        # according to LightGCN paper, this gives better performance
        nn.init.normal_(self.users_emb.weight, std=0.1)
        nn.init.normal_(self.items_emb.weight, std=0.1)

        # create a linear layer (fully connected layer) so we can output a single value (predicted_rating)
        self.out = nn.Linear(embedding_dim + embedding_dim, 1)

    def forward(self, edge_index: Tensor, edge_values: Tensor):
        """Forward propagation of LightGCN Model.

        Args:
            edge_index (SparseTensor): adjacency matrix

        Returns:
            tuple (Tensor): e_u_k, e_u_0, e_i_k, e_i_0
        """

        """
            compute \tilde{A}: symmetrically normalized adjacency matrix
            \tilde_A = D^(-1/2) * A * D^(-1/2)    according to LightGCN paper

            this is essentially a metrix operation way to get 1/ (sqrt(n_neighbors_i) * sqrt(n_neighbors_j))


            if your original edge_index look like
            tensor([[   0,    0,    0,  ...,  609,  609,  609],
                    [   0,    2,    5,  ..., 9444, 9445, 9485]])

                    torch.Size([2, 99466])

            then this will output:
                (
                 tensor([[   0,    0,    0,  ...,  609,  609,  609],
                         [   0,    2,    5,  ..., 9444, 9445, 9485]]),
                 tensor([0.0047, 0.0096, 0.0068,  ..., 0.0592, 0.0459, 0.1325])
                 )

              where edge_index_norm[0] is just the original edge_index

              and edge_index_norm[1] is the symmetrically normalization term.

            under the hood it's basically doing
                def compute_gcn_norm(edge_index, emb):
                    emb = emb.weight
                    from_, to_ = edge_index
                    deg = degree(to_, emb.size(0), dtype=emb.dtype)
                    deg_inv_sqrt = deg.pow(-0.5)
                    deg_inv_sqrt[deg_inv_sqrt == float('inf')] = 0
                    norm = deg_inv_sqrt[from_] * deg_inv_sqrt[to_]

                    return norm


        """
        edge_index_norm = gcn_norm(edge_index=edge_index,
                                   add_self_loops=self.add_self_loops)

        # concat the user_emb and item_emb as the layer0 embing matrix
        # size will be (n_users + n_items) x emb_vector_len.   e.g: 10334 x 64
        emb_0 = torch.cat([self.users_emb.weight, self.items_emb.weight]) # E^0

        embs = [emb_0] # save the layer0 emb to the embs list

        # emb_k is the emb that we are actually going to push it through the graph layers
        # as described in lightGCN paper formula 7
        emb_k = emb_0

        # push the embedding of all users and items through the Graph Model K times.
        # K here is the number of layers
        for i in range(self.K):
            emb_k = self.propagate(edge_index=edge_index_norm[0], x=emb_k, norm=edge_index_norm[1])
            embs.append(emb_k)


        # this is doing the formula8 in LightGCN paper

        # the stacked embs is a list of embedding matrix at each layer
        #    it's of shape n_nodes x (n_layers + 1) x emb_vector_len.
        #        e.g: torch.Size([10334, 4, 64])
        embs = torch.stack(embs, dim=1)

        # From LightGCn paper: "In our experiments, we find that setting α_k uniformly as 1/(K + 1)
        #    leads to good performance in general."
        emb_final = torch.mean(embs, dim=1) # E^K

        users_emb_final, items_emb_final = torch.split(emb_final,
                                                       [self.num_users, self.num_items]) # splits into e_u^K and e_i^K


        r_mat_edge_index, _ = convert_adj_mat_edge_index_to_r_mat_edge_index(edge_index, edge_values)

        src, dest =  r_mat_edge_index[0], r_mat_edge_index[1]

        # applying embedding lookup to get embeddings for src nodes and dest nodes in the edge list
        user_embeds = users_emb_final[src]
        item_embeds = items_emb_final[dest]

        # output dim: edge_index_len x 128 (given 64 is the original emb_vector_len)
        output = torch.cat([user_embeds, item_embeds], dim=1)

        # push it through the linear layer
        output = self.out(output)

        return output

    def message(self, x_j, norm):
        return norm.view(-1, 1) * x_j

layers = 5
model = LightGCN(num_users=num_users,
                 num_items=num_books,
                 K=layers)

In [ ]:

# Dont run here. this part is implemented in the next notebook(Graph based titled...)
from google.colab import files

# Use files.upload() to upload a file from your local system
uploaded = files.upload()

map_location=torch.device('cpu')
# Load the uploaded model file
model_path = 'trained_graph_rec_model.pth'  # Specify the filename
model = torch.load(model_path,map_location)


Saving trained_graph_rec_model.pth to trained_graph_rec_model (1).pth


Training

In [ ]:
# define contants
ITERATIONS = 10000
EPOCHS = 10

BATCH_SIZE = 1024

LR = 1e-3
ITERS_PER_EVAL = 200
ITERS_PER_LR_DECAY = 200
K = 10
LAMBDA = 1e-6


In [ ]:
print(f"BATCH_SIZE {BATCH_SIZE}")

BATCH_SIZE 1024


In [ ]:
# setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device {device}.")


model = model.to(device)
model.train()

# add decay to avoid overfit
optimizer = optim.Adam(model.parameters(), lr=LR, weight_decay=0.01)

scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=1.02)

edge_index = edge_index.to(device)
train_edge_index = train_edge_index.to(device)
val_edge_index = val_edge_index.to(device)


loss_func = nn.MSELoss()

Using device cuda.


In [ ]:
def get_recall_at_k(input_edge_index,
                     input_edge_values, # the true label of actual ratings for each user/item interaction
                     pred_ratings, # the list of predicted ratings
                     k=10,
                     threshold=3.5):
    with torch.no_grad():
        user_item_rating_list = defaultdict(list)

        for i in range(len(input_edge_index[0])):
            src = input_edge_index[0][i].item()
            dest = input_edge_index[1][i].item()
            true_rating = input_edge_values[i].item()
            pred_rating = pred_ratings[i].item()

            user_item_rating_list[src].append((pred_rating, true_rating))

        recalls = dict()
        precisions = dict()

        for user_id, user_ratings in user_item_rating_list.items():
            user_ratings.sort(key=lambda x: x[0], reverse=True)

            n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)

            n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])

            n_rel_and_rec_k = sum(
                ((true_r >= threshold) and (est >= threshold))
                for (est, true_r) in user_ratings[:k]
            )

            precisions[user_id] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 0
            recalls[user_id] = n_rel_and_rec_k / n_rel if n_rel != 0 else 0

        overall_recall = sum(rec for rec in recalls.values()) / len(recalls)
        overall_precision = sum(prec for prec in precisions.values()) / len(precisions)

        return overall_recall, overall_precision


In [ ]:
train_edge_index

In [ ]:
r_mat_train_edge_index, r_mat_train_edge_values = convert_adj_mat_edge_index_to_r_mat_edge_index(train_edge_index, train_edge_values)
r_mat_val_edge_index, r_mat_val_edge_values = convert_adj_mat_edge_index_to_r_mat_edge_index(val_edge_index, val_edge_values)
r_mat_test_edge_index, r_mat_test_edge_values = convert_adj_mat_edge_index_to_r_mat_edge_index(test_edge_index, test_edge_values)

In [ ]:
# training loop
train_losses = []
val_losses = []
val_recall_at_ks = []

for iter in tqdm(range(ITERATIONS)):
    # forward propagation

    # the rating here is based on r_mat
    pred_ratings = model.forward(train_edge_index, train_edge_values)


    train_loss = loss_func(pred_ratings, r_mat_train_edge_values.view(-1,1))


    optimizer.zero_grad()
    train_loss.backward()
    optimizer.step()

    # going over validation set
    if iter % ITERS_PER_EVAL == 0:
        model.eval()

        with torch.no_grad():
            val_pred_ratings = model.forward(val_edge_index, val_edge_values)

            val_loss = loss_func(val_pred_ratings, r_mat_val_edge_values.view(-1,1)).sum()

            recall_at_k, precision_at_k = get_recall_at_k(r_mat_val_edge_index,
                                                          r_mat_val_edge_values,
                                                          val_pred_ratings,
                                                          k = 20
                                                         )


            val_recall_at_ks.append(round(recall_at_k, 5))
            train_losses.append(train_loss.item())
            val_losses.append(val_loss.item())

            print(f"[Iteration {iter}/{ITERATIONS}], train_loss: {round(train_loss.item(), 5)}, val_loss: {round(val_loss.item(), 5)},  recall_at_k {round(recall_at_k, 5)}, precision_at_k {round(precision_at_k, 5)}")

        model.train()

    if iter % ITERS_PER_LR_DECAY == 0 and iter != 0:
        scheduler.step()

  0%|          | 0/10000 [00:00<?, ?it/s]

[Iteration 0/10000], train_loss: 16.54942, val_loss: 16.52435,  recall_at_k 0.0, precision_at_k 0.0
[Iteration 200/10000], train_loss: 4.75522, val_loss: 4.81642,  recall_at_k 0.0, precision_at_k 0.0
[Iteration 400/10000], train_loss: 2.65638, val_loss: 2.69014,  recall_at_k 0.17672, precision_at_k 0.36631
[Iteration 600/10000], train_loss: 2.16042, val_loss: 2.17222,  recall_at_k 0.24596, precision_at_k 0.43536
[Iteration 800/10000], train_loss: 1.90242, val_loss: 1.90067,  recall_at_k 0.28103, precision_at_k 0.4627
[Iteration 1000/10000], train_loss: 1.73559, val_loss: 1.72532,  recall_at_k 0.31374, precision_at_k 0.48645
[Iteration 1200/10000], train_loss: 1.61604, val_loss: 1.60048,  recall_at_k 0.34448, precision_at_k 0.50609
[Iteration 1400/10000], train_loss: 1.52489, val_loss: 1.50621,  recall_at_k 0.37088, precision_at_k 0.52297
[Iteration 1600/10000], train_loss: 1.45243, val_loss: 1.43225,  recall_at_k 0.39255, precision_at_k 0.53558
[Iteration 1800/10000], train_loss: 1.393

Recommendation Function

In [ ]:
def recommend_books(user_id, model, lbl_books, num_recommendations, device):
    # Assuming 'user_id' is the user for whom you want personalized recommendations
    # user_embedding is a learned vector that represents the user representation
    user_embedding = model.users_emb(torch.tensor(user_id).to(device))



    # Get all item embeddings
    all_item_embeddings = model.items_emb(torch.arange(len(lbl_books.classes_)).to(device))

    # print("all_item_embeddings:  ",all_item_embeddings)

    # Compute scores for all items using dot product
    scores = torch.mm(all_item_embeddings, user_embedding.view(-1, 1)).squeeze().cpu().detach().numpy()

    # print("scores: ",scores)

    # Sort the items by their scores in descending order
    sorted_indices = np.argsort(scores, axis=0)[::-1]

    # print("sorted_indices: ",sorted_indices)

    # # Get the top 'num_recommendations' book indices
    top_indices = sorted_indices[:num_recommendations]

    # # Map the book indices back to their original labels
    recommended_books = lbl_books.classes_[top_indices]

    # Create a list of tuples containing book and rating
    recommendations_with_ratings = [(book, score) for book, score in zip(recommended_books, scores[top_indices])]
    print("ratings: ",recommendations_with_ratings)

    return recommended_books,recommendations_with_ratings

In [ ]:
user_id = 87
num_recommendations = 5

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

recommended_books,recommendations_with_ratings = recommend_books(user_id, model, lbl_books, num_recommendations, device)

print(f"Top {num_recommendations} recommended books with ratings for user {user_id}:")
for i, (book_id, rating) in enumerate(zip(recommended_books, recommendations_with_ratings)):
    book_info = books[books['book_id'] == book_id]
    title = book_info['title'].values[0]
    # print(f"{rating[1]:.16f}")
    print(f"{i+1}. Book ID: {book_id}, Title: {title}")

ratings:  [(590, 1.4748727e-07), (69, 1.3807008e-07), (30, 1.0752286e-07), (425, 8.411688e-08), (733, 7.826092e-08)]
Top 5 recommended books with ratings for user 87:
1. Book ID: 590, Title: Lucene in Action
2. Book ID: 69, Title: Ain't No Tomorrow
3. Book ID: 30, Title: Earth Magic: A Book of Shadows for Positive Witches
4. Book ID: 425, Title: Why Civil Resistance Works: The Strategic Logic of Nonviolent Conflict
5. Book ID: 733, Title: As a Driven Leaf


In [ ]:
from google.colab import files

# Save the model to a file
model_path = 'trained_graph_rec_model.pth'  # Specify the path and filename
lbl_books_file_path = 'label_encoder_books.pkl'
torch.save(model.state_dict(), model_path)

# Optionally, save other relevant information (e.g., LabelEncoder)
# For example, to save lbl_books:
import joblib
joblib.dump(lbl_books, lbl_books_file_path)

# Use the files.download function to download the file
files.download(model_path)
files.download(lbl_books_file_path)



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Function to calculate RMSE and MAE
def calculate_rmse_mae(true_ratings, predicted_ratings):
    rmse = np.sqrt(mean_squared_error(true_ratings, predicted_ratings))
    mae = mean_absolute_error(true_ratings, predicted_ratings)
    return rmse, mae

# Function to calculate precision and recall
def calculate_precision_recall(true_ratings, predicted_ratings, threshold=3.5):
    true_positive = sum((true_r >= threshold) and (est >= threshold) for est, true_r in zip(predicted_ratings, true_ratings))
    predicted_positive = sum(est >= threshold for est in predicted_ratings)
    actual_positive = sum(true_r >= threshold for true_r in true_ratings)

    precision = true_positive / predicted_positive if predicted_positive > 0 else 0
    recall = true_positive / actual_positive if actual_positive > 0 else 0

    return precision, recall

# Assuming you have true_ratings and predicted_ratings (e.g., from your validation set)
true_ratings = r_mat_val_edge_values.cpu().detach().numpy().flatten()
predicted_ratings = val_pred_ratings.cpu().detach().numpy().flatten()

# Calculate RMSE and MAE
rmse, mae = calculate_rmse_mae(true_ratings, predicted_ratings)
print(f"RMSE: {rmse:.4f}")
print(f"MAE: {mae:.4f}")

# Calculate precision and recall
precision, recall = calculate_precision_recall(true_ratings, predicted_ratings, threshold=3.5)
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")


RMSE: 1.0266
MAE: 0.8247
Precision: 0.6682
Recall: 1.0000
